#Smooth Data

In [7]:
import pandas as pd

file_path = "./Physionet_ECGs/sel_100_tabsep.csv"
data = pd.read_csv(file_path, sep = '\t', index_col= 0, header=None)
data.head()

,1,2
0,,
0.000,4.725,4.775
0.004,4.735,4.745
0.008,4.725,4.735
0.012,4.715,4.745
0.016,4.720,4.735


Select the column you want to smooth

In [50]:
import numpy as np
import matplotlib.pyplot as plt
col = 1 #choose 1 or 2
values = data[col].values
times = data[col].index

##Smooth Data With Fast Fourier Transform

Fast Fourier Transform and plot the data

In [51]:
from numpy import fft
transformed = fft.fft(values)
real = transformed.real
plt.plot(range(len(real)),real)
plt.ylim(1000,-1000)
plt.show()

Mask the noise in the FFT'ed data and plot the signal with the mask

In [52]:
def mask_fft(transformed, masks=[(54000,171000)], ylims=(1000,-1000)):
    '''
    The default settings work well for column 1
    '''
    masked = transformed
    for i in masks:
        mask_min = i[0]
        mask_max = i[1]
        masked[mask_min:mask_max] = 0 + 0j
    real2 = transformed.real
    plt.plot(range(len(real2)),real2)
    plt.ylim(*ylims)
    return masked
masked = mask_fft(transformed, [(55000,160000)])
plt.gca()
plt.show()

reverse FFT the signal and plot the smoothed data

In [53]:
ifft = fft.ifft(transformed)
plt.plot(times,data)
plt.plot(times,ifft.real,"r")
plt.legend()
plt.xlim(0,1000)
plt.show()

Save!

##Moving Average

In [15]:
%load_ext rmagic  

/Users/morganfine-morris/anaconda/lib/python2.7/site-packages/IPython/extensions/rmagic.py:693: UserWarning: The rmagic extension in IPython is deprecated in favour of rpy2.ipython. If available, that will be loaded instead.
http://rpy.sourceforge.net/
  warnings.warn("The rmagic extension in IPython is deprecated in favour of "


In [54]:
%%R -i values -o smoothed
ma5 = c(1, 1, 1, 1, 1)/5;
smoothed =filter(values,ma5)
print(smoothed[0:100])

  [1]    NA    NA 4.724 4.720 4.713 4.706 4.705 4.703 4.701 4.699 4.703 4.700
 [13] 4.699 4.700 4.704 4.704 4.705 4.705 4.709 4.714 4.720 4.726 4.736 4.745
 [25] 4.754 4.762 4.772 4.782 4.791 4.795 4.800 4.808 4.811 4.808 4.807 4.806
 [37] 4.801 4.797 4.796 4.799 4.800 4.799 4.799 4.801 4.799 4.795 4.793 4.793
 [49] 4.794 4.793 4.791 4.791 4.788 4.783 4.778 4.779 4.779 4.776 4.771 4.770
 [61] 4.770 4.769 4.765 4.765 4.762 4.758 4.752 4.750 4.751 4.754 4.750 4.749
 [73] 4.751 4.751 4.750 4.751 4.753 4.754 4.753 4.749 4.748 4.751 4.752 4.753
 [85] 4.755 4.762 4.769 4.773 4.777 4.790 4.798 4.803 4.810 4.820 4.824 4.829
 [97] 4.831 4.840 4.848 4.855


In [56]:
df = pd.DataFrame(data = np.array(smoothed), index = times, columns=[col])
df_clean = df.dropna()
plt.plot(df_clean.index, df_clean.values)
plt.plot(times, values, 'r')
plt.show()

In [59]:
df_clean.to_csv('./Physionet_ECGs/sel_100_tabsep_mvsmooth.csv', header=False, sep="\t")